In [2]:
#安裝套件
!pip install tensorflow
!pip install keras

In [3]:
# 掛載雲端硬碟
from google.colab import drive

drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [4]:
# 檢視雲端硬碟資料
!ls "/content/drive/My Drive/Colab Notebooks/EarlyWarningStocksGAN/EWSGAN/"

companylist.csv  index_train  punish.csv   stock_data	       X_scaler  X_train  YC_train  Y_test
index_test	 models       punish.xlsx  stock_data_Fourier  X_test	 YC_test  Y_scaler  Y_train


In [2]:
# import 套件
import os
import numpy as np
import pandas as pd
from numpy import *
from sklearn.preprocessing import MinMaxScaler
import pickle
from pickle import dump

In [3]:
#global variable
googlepath = "drive/MyDrive/Colab Notebooks/EarlyWarningStocksGAN/EWSGAN"
get_last_days = 182 #30
files = os.listdir(f'{googlepath}/stock_data_Fourier')
print(files)

['1236_1101119_Fourier.csv', '1307_1090506_Fourier.csv', '1325_1090204_Fourier.csv', '1342_1111208_Fourier.csv', '1414_1091110_Fourier.csv', '1423_1090623_Fourier.csv', '1432_1090820_Fourier.csv', '1432_1110323_Fourier.csv', '1438_1100408_Fourier.csv', '1441_1100505_Fourier.csv', '1444_1090810_Fourier.csv', '1449_1110819_Fourier.csv', '1453_1110411_Fourier.csv', '1455_1100603_Fourier.csv', '1456_1091127_Fourier.csv', '1460_1090410_Fourier.csv', '1466_1090603_Fourier.csv', '1467_1090608_Fourier.csv', '1468_1090616_Fourier.csv', '1471_1100412_Fourier.csv', '1472_1090320_Fourier.csv', '1474_1090619_Fourier.csv', '1475_1100518_Fourier.csv', '1517_1100407_Fourier.csv', '1524_1110524_Fourier.csv', '1526_1090526_Fourier.csv', '1529_1110207_Fourier.csv', '1533_1100114_Fourier.csv', '1539_1110530_Fourier.csv', '1568_1110526_Fourier.csv', '1598_1090824_Fourier.csv', '1608_1100421_Fourier.csv', '1611_1101104_Fourier.csv', '1616_1100422_Fourier.csv', '1701_1111216_Fourier.csv', '1721_1101015_Fouri

In [4]:
# 新增資料夾
if not os.path.exists(f'{googlepath}/X_scaler'):
    os.makedirs(f'{googlepath}/X_scaler')
if not os.path.exists(f'{googlepath}/Y_scaler'):
    os.makedirs(f'{googlepath}/Y_scaler')
if not os.path.exists(f'{googlepath}/index_test'):
    os.makedirs(f'{googlepath}/index_test')
if not os.path.exists(f'{googlepath}/index_train'):
    os.makedirs(f'{googlepath}/index_train')
if not os.path.exists(f'{googlepath}/X_test'):
    os.makedirs(f'{googlepath}/X_test')
if not os.path.exists(f'{googlepath}/X_train'):
    os.makedirs(f'{googlepath}/X_train')
if not os.path.exists(f'{googlepath}/Y_test'):
    os.makedirs(f'{googlepath}/Y_test')
if not os.path.exists(f'{googlepath}/Y_train'):
    os.makedirs(f'{googlepath}/Y_train')
if not os.path.exists(f'{googlepath}/YC_test'):
    os.makedirs(f'{googlepath}/YC_test')
if not os.path.exists(f'{googlepath}/YC_train'):
    os.makedirs(f'{googlepath}/YC_train')

In [7]:
for filename in files:
  print(filename)
  # 因為在colab執行時會產生.ipynb_checkpoints，執行時會出現error
  if filename != '.ipynb_checkpoints':
    # %% --------------------------------------- Load Data  -----------------------------------------------------------------
    dataset = pd.read_csv(f'{googlepath}/stock_data_Fourier/{filename.split(",")[0]}', parse_dates=['date'])
    # %% --------------------------------------- Data Preprocessing  -----------------------------------------------------------------

    # Replace 0 by NA
    dataset.replace(0, np.nan, inplace=True)

    # Set the date to datetime data
    datetime_series = pd.to_datetime(dataset['date'])
    datetime_index = pd.DatetimeIndex(datetime_series.values)
    dataset = dataset.set_index(datetime_index)
    dataset = dataset.sort_values(by='date')
    y_value = pd.DataFrame(
        (get_last_days - dataset['date_before_announced'] + 1) / get_last_days)  # pd.DataFrame(dataset.iloc[:, 5])
    dataset = dataset.drop(columns=['date','stock_id','AnnouncedDate','spread','date_before_announced']) #,'MeasuringReasonCode'

    # Get features and target
    X_value = pd.DataFrame(dataset.iloc[:, :])

    # Normalized the data
    X_scaler = MinMaxScaler(feature_range=(-1, 1))
    y_scaler = MinMaxScaler(feature_range=(0, 1))
    X_scaler.fit(X_value)
    y_scaler.fit(y_value)

    X_scale_dataset = X_scaler.fit_transform(X_value)
    y_scale_dataset = y_scaler.fit_transform(y_value)
    # test = y_scaler.fit_transform(y_value)
    # y_scale_dataset = y_value
    dump(X_scaler, open(f'{googlepath}/X_scaler/X_scaler{filename.split(",")[0].split(".")[0]}.pkl', 'wb'))
    dump(y_scaler, open(f'{googlepath}/Y_scaler/y_scaler{filename.split(",")[0].split(".")[0]}.pkl', 'wb'))

    # Reshape the data
    n_steps_in = 5
    n_features = X_value.shape[1]
    n_steps_out = 1

    # Get X/y dataset
    def get_X_y(X_data, y_data):
      X = list()
      y = list()
      yc = list()

      length = len(X_data)
      for i in range(0, length, 1):
        X_value = X_data[i: i + n_steps_in][:, :]
        y_value = y_data[i + n_steps_in: i + (n_steps_in + n_steps_out)][:, 0]
        if np.isnan(y_value):
          print("y_Value is Nan")
        yc_value = y_data[i: i + n_steps_in][:, :]
        if len(X_value) == n_steps_in and len(y_value) == n_steps_out:
          X.append(X_value)
          y.append(y_value)
          yc.append(yc_value)
      return np.array(X), np.array(y), np.array(yc)

    # get the train test predict index
    def predict_index(dataset, X_train, n_steps_in, n_steps_out):
      train_predict_index = dataset.iloc[n_steps_in: X_train.shape[0] + n_steps_in + n_steps_out - 1, :].index
      test_predict_index = dataset.iloc[X_train.shape[0] + n_steps_in:, :].index
      return train_predict_index, test_predict_index

    # Split train/test dataset
    def split_train_test(data):
      train_size = round(len(X) * 0.7)
      data_train = data[0:train_size]
      data_test = data[train_size:]
      return data_train, data_test

    # Get data and check shape
    X, y, yc = get_X_y(X_scale_dataset, y_scale_dataset)
    X_train, X_test, = split_train_test(X)
    y_train, y_test, = split_train_test(y)
    yc_train, yc_test, = split_train_test(yc)
    index_train, index_test, = predict_index(dataset, X_train, n_steps_in, n_steps_out)
    # %% --------------------------------------- Save dataset -----------------------------------------------------------------
    print('X shape: ', X.shape)
    print('y shape: ', y.shape)
    print('X_train shape: ', X_train.shape)
    print('y_train shape: ', y_train.shape)
    print('y_c_train shape: ', yc_train.shape)
    print('X_test shape: ', X_test.shape)
    print('y_test shape: ', y_test.shape)
    print('y_c_test shape: ', yc_test.shape)
    print('index_train shape:', index_train.shape)
    print('index_test shape:', index_test.shape)

    np.save(f'{googlepath}/X_train/X_train{filename.split(",")[0].split(".")[0]}.npy', X_train)
    np.save(f'{googlepath}/Y_train/y_train{filename.split(",")[0].split(".")[0]}.npy', y_train)
    np.save(f'{googlepath}/X_test/X_test{filename.split(",")[0].split(".")[0]}.npy', X_test)
    np.save(f'{googlepath}/Y_test/y_test{filename.split(",")[0].split(".")[0]}.npy', y_test)
    np.save(f'{googlepath}/YC_train/yc_train{filename.split(",")[0].split(".")[0]}.npy', yc_train)
    np.save(f'{googlepath}/YC_test/yc_test{filename.split(",")[0].split(".")[0]}.npy', yc_test)
    np.save(f'{googlepath}/index_train/index_train{filename.split(",")[0].split(".")[0]}.npy', index_train)
    np.save(f'{googlepath}/index_test/index_test{filename.split(",")[0].split(".")[0]}.npy', index_test)

1236_1101119_Fourier.csv
X shape:  (177, 5, 25)
y shape:  (177, 1)
X_train shape:  (124, 5, 25)
y_train shape:  (124, 1)
y_c_train shape:  (124, 5, 1)
X_test shape:  (53, 5, 25)
y_test shape:  (53, 1)
y_c_test shape:  (53, 5, 1)
index_train shape: (124,)
index_test shape: (53,)
1307_1090506_Fourier.csv


<ipython-input-7-32035a2236e4>:52: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if np.isnan(y_value):
<ipython-input-7-32035a2236e4>:52: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if np.isnan(y_value):


X shape:  (177, 5, 25)
y shape:  (177, 1)
X_train shape:  (124, 5, 25)
y_train shape:  (124, 1)
y_c_train shape:  (124, 5, 1)
X_test shape:  (53, 5, 25)
y_test shape:  (53, 1)
y_c_test shape:  (53, 5, 1)
index_train shape: (124,)
index_test shape: (53,)
1325_1090204_Fourier.csv


<ipython-input-7-32035a2236e4>:52: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if np.isnan(y_value):


X shape:  (177, 5, 25)
y shape:  (177, 1)
X_train shape:  (124, 5, 25)
y_train shape:  (124, 1)
y_c_train shape:  (124, 5, 1)
X_test shape:  (53, 5, 25)
y_test shape:  (53, 1)
y_c_test shape:  (53, 5, 1)
index_train shape: (124,)
index_test shape: (53,)
1342_1111208_Fourier.csv


...

X shape:  (177, 5, 25)
y shape:  (177, 1)
X_train shape:  (124, 5, 25)
y_train shape:  (124, 1)
y_c_train shape:  (124, 5, 1)
X_test shape:  (53, 5, 25)
y_test shape:  (53, 1)
y_c_test shape:  (53, 5, 1)
index_train shape: (124,)
index_test shape: (53,)
9928_1110825_Fourier.csv
X shape:  (177, 5, 25)
y shape:  (177, 1)
X_train shape:  (124, 5, 25)
y_train shape:  (124, 1)
y_c_train shape:  (124, 5, 1)
X_test shape:  (53, 5, 25)
y_test shape:  (53, 1)
y_c_test shape:  (53, 5, 1)
index_train shape: (124,)
index_test shape: (53,)
9935_1090319_Fourier.csv
X shape:  (177, 5, 25)
y shape:  (177, 1)
X_train shape:  (124, 5, 25)
y_train shape:  (124, 1)
y_c_train shape:  (124, 5, 1)
X_test shape:  (53, 5, 25)
y_test shape:  (53, 1)
y_c_test shape:  (53, 5, 1)
index_train shape: (124,)
index_test shape: (53,)


<ipython-input-7-32035a2236e4>:52: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if np.isnan(y_value):
<ipython-input-7-32035a2236e4>:52: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if np.isnan(y_value):


9944_1090204_Fourier.csv
X shape:  (177, 5, 25)
y shape:  (177, 1)
X_train shape:  (124, 5, 25)
y_train shape:  (124, 1)
y_c_train shape:  (124, 5, 1)
X_test shape:  (53, 5, 25)
y_test shape:  (53, 1)
y_c_test shape:  (53, 5, 1)
index_train shape: (124,)
index_test shape: (53,)
9955_1110224_Fourier.csv


<ipython-input-7-32035a2236e4>:52: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if np.isnan(y_value):
<ipython-input-7-32035a2236e4>:52: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if np.isnan(y_value):


X shape:  (177, 5, 25)
y shape:  (177, 1)
X_train shape:  (124, 5, 25)
y_train shape:  (124, 1)
y_c_train shape:  (124, 5, 1)
X_test shape:  (53, 5, 25)
y_test shape:  (53, 1)
y_c_test shape:  (53, 5, 1)
index_train shape: (124,)
index_test shape: (53,)


In [12]:
#因為部分檔案會造成執行時出現nan，所以先個別檔案跑一次，如果出現nan，則予以排除

from pathlib import Path
import time
import os
import numpy as np
import tensorflow as tf
from math import sqrt
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import GRU
from keras.optimizers import Adam
from matplotlib import pyplot
for file_name in os.listdir(f'{googlepath}/X_train'):
  print(file_name)
  file_path_X_train = Path(f"{googlepath}/X_train/{file_name}")
  file_path_X_test = Path(f"{googlepath}/X_test/{file_name.replace('X_train','X_test')}")
  file_path_Y_train = Path(f"{googlepath}/Y_train/{file_name.replace('X_train','y_train')}")
  file_path_Y_test = Path(f"{googlepath}/Y_test/{file_name.replace('X_train','y_test')}")
  file_path_YC_train = Path(f"{googlepath}/YC_train/{file_name.replace('X_train','yc_train')}")
  file_path_YC_test = Path(f"{googlepath}/YC_test/{file_name.replace('X_train','yc_test')}")
  file_path_X_scaler = Path(f"{googlepath}/X_scaler/{file_name.replace('X_train','X_scaler').replace('npy','pkl')}")
  print(file_path_X_scaler)
  file_path_Y_scaler = Path(f"{googlepath}/Y_scaler/{file_name.replace('X_train','y_scaler').replace('npy','pkl')}")
  print(file_path_Y_scaler)
  file_path_index_test = Path(f"{googlepath}/index_test/{file_name.replace('X_train','index_test')}")
  print(file_path_index_test)
  file_path_index_train = Path(f"{googlepath}/index_train/{file_name.replace('X_train','index_train')}")
  print(file_path_index_train)
  file_path_stock_data = Path(f"{googlepath}/stock_data/{file_name.replace('X_train','').replace('_Fourier','').replace('npy','csv')}")
  print(file_path_stock_data)
  file_path_stock_data_Fourier = Path(f"{googlepath}/stock_data_Fourier/{file_name.replace('X_train','').replace('npy','csv')}")
  print(file_path_stock_data_Fourier)
  print(file_path_X_train)
  print(file_path_X_train.name)
  if file_path_X_train.name in [
      "X_train1423_1090623_Fourier.npy",
      "X_train1438_1100408_Fourier.npy",
      "X_train1466_1090603_Fourier.npy",
      "X_train1467_1090608_Fourier.npy",
      "X_train1472_1090320_Fourier.npy",
      "X_train1475_1100518_Fourier.npy",
      "X_train1732_1090203_Fourier.npy",
      "X_train1734_1090708_Fourier.npy",
      "X_train1734_1100809_Fourier.npy",
      "X_train1760_1100520_Fourier.npy",
      "X_train1760_1110420_Fourier.npy",
      "X_train1903_1090710_Fourier.npy",
      "X_train2014_1100412_Fourier.npy",
      "X_train2024_1100702_Fourier.npy",
      "X_train2028_1091118_Fourier.npy",
      "X_train2028_1100604_Fourier.npy",
      "X_train2364_1110606_Fourier.npy",
      "X_train2390_1090622_Fourier.npy",
      "X_train2424_1090611_Fourier.npy",
      "X_train2429_1090115_Fourier.npy",
      "X_train2489_1100323_Fourier.npy",
      "X_train2540_1111011_Fourier.npy",
      "X_train2712_1090529_Fourier.npy",
      "X_train2722_1090601_Fourier.npy",
      "X_train3025_1091202_Fourier.npy",
      "X_train3060_1100426_Fourier.npy",
      "X_train3138_1100108_Fourier.npy",
      "X_train3308_1100412_Fourier.npy",
      "X_train3669_1090810_Fourier.npy",
      "X_train3682_1090914_Fourier.npy",
      "X_train3701_1100907_Fourier.npy",
      "X_train3712_1090623_Fourier.npy",
      "X_train4414_1090619_Fourier.npy",
      "X_train4770_1110826_Fourier.npy",
      "X_train4960_1090407_Fourier.npy",
      "X_train6172_1101029_Fourier.npy",
      "X_train6531_1090504_Fourier.npy",
      "X_train6698_1090320_Fourier.npy",
      "X_train6756_1100621_Fourier.npy",
      "X_train6796_1110713_Fourier.npy",
      "X_train9928_1110825_Fourier.npy",
      "X_train9944_1090204_Fourier.npy"]:
    print(f'刪除相關檔案: {file_path_X_train}')

    os.remove(file_path_X_train)
    os.remove(file_path_X_test)
    os.remove(file_path_Y_train)
    os.remove(file_path_Y_test)
    os.remove(file_path_YC_train)
    os.remove(file_path_YC_test)
    os.remove(file_path_X_scaler)
    os.remove(file_path_Y_scaler)
    os.remove(file_path_index_test)
    os.remove(file_path_index_train)
    os.remove(file_path_stock_data)
    os.remove(file_path_stock_data_Fourier)
  # if not (file_path_stock_data.exists() and file_path_X_test.exists() and file_path_Y_train.exists() and file_path_Y_test.exists() and file_path_YC_train.exists() and file_path_YC_test.exists()):
  #   # and file_path_stock_data_Fourier.exists() and file_path_X_scaler.exists() and file_path_Y_scaler.exists() and file_path_index_test.exists() and file_path_index_train.exists()
  #   print(f'其他檔案不存在: {file_name}')
  #   time.sleep(20)
  #   os.remove(file_path_X_train)
  #   os.remove(file_path_X_test)
  #   os.remove(file_path_Y_train)
  #   os.remove(file_path_Y_test)
  #   os.remove(file_path_YC_train)
  #   os.remove(file_path_YC_test)
  #   os.remove(file_path_X_scaler)
  #   os.remove(file_path_Y_scaler)
  #   os.remove(file_path_index_test)
  #   os.remove(file_path_index_train)
  #   os.remove(file_path_stock_data)
  #   os.remove(file_path_stock_data_Fourier)
  #   break
  # else:
  #   print(f"其他檔案皆存在{file_name}")

  # # Load data
  # X_train = np.load(file_path_X_train, allow_pickle=True)
  # X_test = np.load(file_path_X_test, allow_pickle=True)
  # y_train = np.load(file_path_Y_train, allow_pickle=True)
  # y_test = np.load(file_path_Y_test, allow_pickle=True)
  # yc_train = np.load(file_path_YC_train, allow_pickle=True)
  # yc_test = np.load(file_path_YC_test, allow_pickle=True)
  # print('X_train shape: ', X_train.shape)
  # print('y_train shape: ', y_train.shape)
  # print('y_c_train shape: ', yc_train.shape)
  # print('X_test shape: ', X_test.shape)
  # print('y_test shape: ', y_test.shape)
  # print('y_c_test shape: ', yc_test.shape)

  # # Parameters
  # LR = 0.0001
  # BATCH_SIZE = 32
  # N_EPOCH = 1

  # input_dim = X_train.shape[1]
  # feature_size = X_train.shape[2]
  # output_dim = y_train.shape[1]

  # def basic_GRU(input_dim, output_dim, feature_size) -> tf.keras.models.Model:
  #   try:
  #     model = Sequential()
  #     model.add(GRU(units=128, return_sequences = True, input_shape=(input_dim, feature_size)))  # 256, return_sequences = True
  #     # model.add(GRU(units=256, recurrent_dropout = 0.2)) #, return_sequences = True
  #     model.add(GRU(units=64, input_shape=(input_dim, feature_size)))
  #     #model.add(Dense(128))
  #     model.add(Dense(32))
  #     # model.add(Dense(32))
  #     model.add(Dense(units=output_dim))
  #     model.compile(optimizer=Adam(learning_rate=LR), loss='mse')
  #     history = model.fit(X_train, y_train, epochs=N_EPOCH, batch_size=BATCH_SIZE, validation_data=(X_test, y_test),
  #                         verbose=2, shuffle=False)
  #     # pyplot.plot(history.history['loss'], label='train')
  #     # pyplot.plot(history.history['val_loss'], label='validation')
  #     # pyplot.legend()
  #     # pyplot.show()
  #     return model
  #   except:
  #     print(f"{file_name}出現異常")
  #     os.remove(file_path_X_train)
  #     os.remove(file_path_X_test)
  #     os.remove(file_path_Y_train)
  #     os.remove(file_path_Y_test)
  #     os.remove(file_path_YC_train)
  #     os.remove(file_path_YC_test)
  #     os.remove(file_path_X_scaler)
  #     os.remove(file_path_Y_scaler)
  #     os.remove(file_path_index_test)
  #     os.remove(file_path_index_train)
  #     os.remove(file_path_stock_data)
  #     os.remove(file_path_stock_data_Fourier)

  # model = basic_GRU(input_dim, output_dim, feature_size)

X_train1236_1101119_Fourier.npy
drive/MyDrive/Colab Notebooks/EarlyWarningStocksGAN/EWSGAN/X_scaler/X_scaler1236_1101119_Fourier.pkl
drive/MyDrive/Colab Notebooks/EarlyWarningStocksGAN/EWSGAN/Y_scaler/y_scaler1236_1101119_Fourier.pkl
drive/MyDrive/Colab Notebooks/EarlyWarningStocksGAN/EWSGAN/index_test/index_test1236_1101119_Fourier.npy
drive/MyDrive/Colab Notebooks/EarlyWarningStocksGAN/EWSGAN/index_train/index_train1236_1101119_Fourier.npy
drive/MyDrive/Colab Notebooks/EarlyWarningStocksGAN/EWSGAN/stock_data/1236_1101119.csv
drive/MyDrive/Colab Notebooks/EarlyWarningStocksGAN/EWSGAN/stock_data_Fourier/1236_1101119_Fourier.csv
drive/MyDrive/Colab Notebooks/EarlyWarningStocksGAN/EWSGAN/X_train/X_train1236_1101119_Fourier.npy
X_train1236_1101119_Fourier.npy
X_train1307_1090506_Fourier.npy
drive/MyDrive/Colab Notebooks/EarlyWarningStocksGAN/EWSGAN/X_scaler/X_scaler1307_1090506_Fourier.pkl
drive/MyDrive/Colab Notebooks/EarlyWarningStocksGAN/EWSGAN/Y_scaler/y_scaler1307_1090506_Fourier.pkl